In [1]:
import pdb

import numpy as np
import pandas as pd

from valkyrie.securities import *

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
%matplotlib notebook
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth',100)

/tmp/ipykernel_11070/1899750729.py:8: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
def build_universe_df(url):
    from html_table_parser.parser import HTMLTableParser
    with open(url, 'r') as f:
        xhtml = f.read()
    p = HTMLTableParser()
    p.feed(xhtml)
    table = p.tables[-2]

    orig_len = len(table)

    table = [r for r in table if len(r) == 12]
    new_len = len(table)
    print(f'remaing table rows : {new_len}/{orig_len}')

    cols = ['SymbolCUSIP', 'des', 'exch', 'IPO', 'cpn', 'LiqPref', 'CallMat', 'rating', 'Tax15', 'Conv', 'Prosp',
      'Distribution Dates']
    df = pd.DataFrame(table[1:], columns=['orig_' + c for c in cols])
    df['symbol'] = df['orig_SymbolCUSIP'].apply(lambda s: substr(s, 0).replace('-', ' PR'))
    df['cusip'] = df['orig_SymbolCUSIP'].apply(lambda s: substr(s, 1))
    df['des'] = df['orig_des']
    df['exch'] = df['orig_exch'].apply(lambda s: substr(s, 0))
    df['ipo'] = df['orig_IPO'].apply(str2ts)
    for c in ['symbol', 'cusip', 'des', 'exch']:
        df[c] = df[c].astype(str)
        
    df['float'] = df['des'].apply(lambda x : 'FLOAT' in x.upper())
    df['parent']= df['symbol'].apply(lambda x : parent(x))
    return df[[c for c in df if 'orig' not in c]].set_index('symbol')#df.drop([c for c in df if 'orig_' in c], axis=1).sort_values('symbol').reset_index().drop('index', axis=1)

In [3]:
url ='/home/bb/projects/valkyrie/data/universe/All-Exchange-Traded-Income-Securities_20220112.html'

In [4]:
df = build_universe_df(url)

remaing table rows : 1059/1064


In [10]:
df.query('parent == "BAC" and float')

,cusip,des,exch,ipo,float,parent
symbol,,,,,,
BAC PRE,060505815,"Bank of America, Floating Rate Dep Shares Non-cumul Preferred Stock, Series E",NYSE,2006-11-01,True,BAC


In [19]:
df.query('parent == "WFC" and not float')

,cusip,des,exch,ipo,float,parent
symbol,,,,,,
WFC PRD,95002Y400,Wells Fargo Co 4.25% Dep Shares Non-Cumul Perp Cls A Preferred Stock Series DD,NYSE,2021-07-20,False,WFC
WFC PRC,95002Y202,"Wells Fargo Co., 4.375% Dep Shares Non-Cumul Class A Preferred Stock Series CC",NYSE,2021-01-25,False,WFC
WFC PRA,94988U128,"Wells Fargo Co., 4.70% Dep Shares Non-Cumul Class A Preferred Stock Series AA",NYSE,2020-10-21,False,WFC
WFC PRZ,94988U151,Wells Fargo Co. 4.75% Dep Shares Non-Cumul Perp Cls A Preferred Stock Series Z,NYSE,2020-01-15,False,WFC
WFC PRY,94988U656,"Wells Fargo Co., 5.625% Dep Shares Non-Cumul Class A Preferred Stock Series Y",NYSE,2017-04-17,False,WFC
WFC PRL,949746804,"Wells Fargo Co., 7.50% Non-Cumul Perp Conv Cl A Preferred Stock, Series L",NYSE,2008-04-14,False,WFC


In [12]:
df.query('parent == "MS" and float')

,cusip,des,exch,ipo,float,parent
symbol,,,,,,
MS PRK,61762V606,"Morgan Stanley, 5.85% Dep Shares Fixed/Float Non-Cumul Preferred Stock Series K",NYSE,2017-01-24,True,MS
MS PRI,61761J406,"Morgan Stanley, 6.375% Dep Shares Fixed/Float Non-Cumul Preferred Stock Series I",NYSE,2014-09-11,True,MS
MS PRF,61763E207,"Morgan Stanley, 6.875% Dep Shares Fixed/Floating Non-Cumul Preferred Stock Ser F",NYSE,2013-12-05,True,MS
MS PRE,61762V200,"Morgan Stanley, 7.125% Dep Shares Fixed/Floating Non-Cumul Preferred Stock Ser E",NYSE,2013-09-24,True,MS
MS PRA,61747S504,Morgan Stanley Floating Rate Dep Shares Series A Non-cumul Preferred Stock,NYSE,2006-06-28,True,MS


In [13]:
df.query('parent == "GS" and float')

,cusip,des,exch,ipo,float,parent
symbol,,,,,,
GS PRK,38148B108,"Goldman Sachs Group, 6.375% Dep Sh Fixd/Float Non-Cumul Preferred Stock Series K",NYSE,2014-04-21,True,GS
GS PRJ,38145G308,"Goldman Sachs Group, 5.50% Dep Shares Fixd/Float Non-Cumul Preferred Stock Ser J",NYSE,2013-04-17,True,GS
GS PRD,38144G804,"Goldman Sachs Group, Dep Shares Float Rate Non-Cumul Preferred Stock, Series D",NYSE,2006-05-18,True,GS
GS PRC,38144X609,"Goldman Sachs Group, Dep Shares Float Rate Non-cumul Preferred Stock, Series C",NYSE,2005-10-25,True,GS
GS PRA,38143Y665,"Goldman Sachs Group, Dep Shares Float Rate Non-cumul Preferred Stock, Series A",NYSE,2005-04-18,True,GS


In [16]:
0.04 / 21.45*25

0.046620046620046623

In [15]:
0.0585

0.0585

In [17]:
4.25 / 20.21 * 25

5.257298367144977

In [18]:
4.00 / 21.57 * 25

4.6360686138154845

In [22]:
(2.86671 + 0.75) / 21.50 * 25

4.205476744186046

In [21]:
 3.64 + 3.86

7.5